In [13]:
pip install ortools

In [14]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import numpy as np

In [33]:
matrix_1 = [
  [0, 2451, 713, 1018, 1631, 1374, 2408, 213, 2571, 875, 1420, 2145, 1972],
  [2451, 0, 1745, 1524, 831, 1240, 959, 2596, 403, 1589, 1374, 357, 579],
  [713, 1745, 0, 355, 920, 803, 1737, 851, 1858, 262, 940, 1453, 1260],
  [1018, 1524, 355, 0, 700, 862, 1395, 1123, 1584, 466, 1056, 1280, 987],
  [1631, 831, 920, 700, 0, 663, 1021, 1769, 949, 796, 879, 586, 371],
  [1374, 1240, 803, 862, 663, 0, 1681, 1551, 1765, 547, 225, 887, 999],
  [2408, 959, 1737, 1395, 1021, 1681, 0, 2493, 678, 1724, 1891, 1114, 701],
  [213, 2596, 851, 1123, 1769, 1551, 2493, 0, 2699, 1038, 1605, 2300, 2099],
  [2571, 403, 1858, 1584, 949, 1765, 678, 2699, 0, 1744, 1645, 653, 600],
  [875, 1589, 262, 466, 796, 547, 1724, 1038, 1744, 0, 679, 1272, 1162],
  [1420, 1374, 940, 1056, 879, 225, 1891, 1605, 1645, 679, 0, 1017, 1200],
  [2145, 357, 1453, 1280, 586, 887, 1114, 2300, 653, 1272, 1017, 0, 504],
  [1972, 579, 1260, 987, 371, 999, 701, 2099, 600, 1162, 1200, 504, 0],
]
dist_function_1 = lambda a, b : matrix_1[a][b]
locations_1 = list(range(len(matrix_1)))
capacities_1 = [100, 100, 100, 100]
demands_1 = [0, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]

matrix_2 = [[0, 1, 1], [1, 0, 1], [1, 1, 0]]
dist_function_2 = lambda a, b : matrix_2[a][b]
locations_2 = list(range(len(matrix_2)))
capacities_2 = [(10, 0), (0, 10)]
demands_2 = [(0, 0), (1, 0), (0, 1)]

# TSP

In [35]:
def tsp(dist_function, locations):  # function takes loc 1 and loc 2 and returns dist between

  def create_dist_matrix(locations):
    matrix = np.zeros((len(locations), len(locations))).tolist()
    for l1 in range(len(matrix)):
      for l2 in range(len(matrix)):
        matrix[l1][l2] = dist_function(locations[l1], locations[l2])
    return matrix
        
  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  dist_matrix = create_dist_matrix(locations)
            
  manager = pywrapcp.RoutingIndexManager(len(dist_matrix), 1, 0)

  routing = pywrapcp.RoutingModel(manager)
  transit_callback_index = routing.RegisterTransitCallback(distance_callback)

  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

  solution = routing.SolveWithParameters(search_parameters)

  visited = []
  index = routing.Start(0)
  while not routing.IsEnd(index):
    visited.append(manager.IndexToNode(index))
    index = solution.Value(routing.NextVar(index))
  visited.append(manager.IndexToNode(index))
  
  return visited, solution.ObjectiveValue()

In [36]:
tsp(dist_function_1, locations_1)

([0, 7, 2, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0], 7293)

# tsp/vrp/cvrp combined general function

In [24]:
def routing(dist_function, locations, no_vehicles=1, capacities=[], demands=[], max_travel_dist=int(1e9), time_limit=-1):

  if time_limit > 0:
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
    search_parameters.time_limit.seconds = time_limit

  if capacities == []:  # if no capacity entered assume infinite capacity
    capacities = [(int(1e9),) for __ in range(no_vehicles)]
  if demands == []:  # if no demand entered assume 1 demand
    demands = [(1,) for __ in range(len(locations))]

  if type(demands[0]) != tuple:
    capacities = [(i,) for i in capacities]
    demands = [(i,) for i in demands]

  assert len(capacities) == no_vehicles
  assert len(demands) == len(locations)

  def create_dist_matrix(locations):
    matrix = np.zeros((len(locations), len(locations))).tolist()
    for l1 in range(len(matrix)):
      for l2 in range(len(matrix)):
        matrix[l1][l2] = dist_function(locations[l1], locations[l2])
    return matrix
        
  def distance_callback(from_index, to_index):
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return dist_matrix[from_node][to_node]

  def demand_callback(from_index, idx):  # callback for material: idx
    from_node = manager.IndexToNode(from_index)
    return demands[from_node][idx]

  def add(a, b):
    return [a[i]+b[i] for i in range(len(a))]

  dist_matrix = create_dist_matrix(locations)
            
  manager = pywrapcp.RoutingIndexManager(len(dist_matrix), no_vehicles, 0)
  routing = pywrapcp.RoutingModel(manager)
  transit_callback_index = routing.RegisterTransitCallback(distance_callback)
  routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

  routing.AddDimension(transit_callback_index, 0, max_travel_dist, True, 'Distance')  # For multiple vehicles
  distance_dimension = routing.GetDimensionOrDie('Distance')
  distance_dimension.SetGlobalSpanCostCoefficient(100)

  for i in range(len(capacities[0])):  # add dimension for each capacity
    demand_callback_index = routing.RegisterUnaryTransitCallback(lambda x : demand_callback(x, i))
    routing.AddDimensionWithVehicleCapacity(demand_callback_index, 0, [j[i] for j in capacities], True, f'Capacity_{i}')

  search_parameters = pywrapcp.DefaultRoutingSearchParameters()
  search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
  solution = routing.SolveWithParameters(search_parameters)

  if not solution:
    raise ValueError("NO SOLUTION :(")

  loads = [[0 for __ in range(len(capacities[0]))] for __ in range(len(capacities))]
  max_route_distance = 0
  visited = [[] for i in range(no_vehicles)]
  for vehicle_id in range(no_vehicles):
      index = routing.Start(vehicle_id)
      route_distance = 0
      while not routing.IsEnd(index):
          visited[vehicle_id].append(manager.IndexToNode(index))
          loads[vehicle_id] = add(loads[vehicle_id], demands[manager.IndexToNode(index)])
          previous_index = index
          index = solution.Value(routing.NextVar(index))
          route_distance += routing.GetArcCostForVehicle(previous_index, index, vehicle_id)
      visited[vehicle_id].append(manager.IndexToNode(index))
      max_route_distance = max(route_distance, max_route_distance)

  return visited, max_route_distance, loads  # route (vehicles, location idx), cost

In [26]:
def tsp(dist_function, locations):
  x = routing(dist_function, locations)
  return x[0][0], x[1]

In [27]:
def vrp(dist_function, locations, no_vehicles):
  x = routing(dist_function, locations, no_vehicles=no_vehicles)
  return x[:2]

In [28]:
def cvrp(dist_function, locations, no_vehicles, capacities, demands):
  x = routing(dist_function, locations, no_vehicles=no_vehicles, capacities=capacities, demands=demands)
  if len(x[2][0]) == 1:
    return x[0], x[1], [i[0] for i in x[2]]
  return x[0], x[1], x[2]

In [29]:
tsp(dist_function_1, locations_1)

([0, 7, 2, 3, 4, 12, 6, 8, 1, 11, 10, 5, 9, 0], 7293)

In [30]:
vrp(dist_function_1, locations_1, 2)

([[0, 10, 5, 11, 1, 4, 2, 7, 0], [0, 3, 6, 8, 12, 9, 0]], 5728)

In [31]:
cvrp(dist_function_1, locations_1, len(capacities_1), capacities_1, demands_1)

([[0, 12, 8, 2, 0], [0, 9, 4, 1, 11, 5, 0], [0, 10, 0], [0, 3, 6, 7, 0]],
 5143,
 [30, 50, 10, 30])

In [34]:
cvrp(dist_function_2, locations_2, len(capacities_2), capacities_2, demands_2)  # WHY DOESNT THIS WORK ??!!!

ValueError: ignored